In [2]:
import libraries_functions
from libraries_functions import * # do not run it - this is the custom library

In [ ]:
# functions from library
def drop_null_columns(df, threshold=0.67):
    """удаляет колонки с процентом незаполненных значений более threshold"""
    df = df.copy()

    temp = pd.Series()
    for col in df.columns:
        temp[col] = df.loc[df.TARGET.notnull(), col].isnull().mean()
    NullField67 = list(temp[temp > threshold].index)
    df = df.drop(NullField67, 1)
    return df


def drop_single_value_columns(df):
    """удаляет колонки с константным значением"""
    df = df.copy()

    SingleValue = []
    for col in df.columns:
        if df[col].nunique() < 2:
            SingleValue.append(col)

    df = df.drop(SingleValue, 1)
    return df


def drop_corr_columns(df, threshold=0.985):
    """Удаляет колонки (одну из пары) с корреляцией более threshold"""
    df = df.copy()

    corr = df.corr()
    to_drop_cor = []
    for i in corr:
        for j in corr.index[corr[i] > threshold]:
            if i != j and j not in to_drop_cor and i not in to_drop_cor:
                to_drop_cor.append(j)

    df = df.drop(to_drop_cor, 1)
    return df

def encode(df, cat_encoding_lim):
    """Кодирует категориальные признаки и сохраняет словарь"""
    df = df.copy()

    d = {}
    obj_cols = df.select_dtypes(include=['object']).columns
    for col in obj_cols:
        enc = cat_encoder(df.loc[df.TARGET.notnull(), col], cat_encoding_lim)
        d[col] = enc
        df[col] = df[col].map(enc)
    return d, df, obj_cols


def cat_encoder(series, n_cats=30):
    """Возвращает словарь для кодирования категориальных признаков, категории присваиваются по убыванию в
    зависимости от частоты категории (если частоты совпадают - в алфавитном порядке)"""
    d = dict(series.value_counts())
    d = sorted(d.items(), key=lambda t: t[::-1], reverse=True)
    d = islice(d, n_cats)
    return {x[0]: int(i) for i, x in enumerate(d)}

In [3]:
import pandas as pd

In [29]:
train = pd.read_csv("input/application_train.csv",sep=',')
test = pd.read_csv("input/application_test.csv",sep=',')

In [34]:
df = train.append(test)

/Users/ilyaboytsov/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [37]:
df = df.reset_index(drop=True)

In [38]:
df.head()

,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,AMT_INCOME_TOTAL,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_YEAR,APARTMENTS_AVG,APARTMENTS_MEDI,APARTMENTS_MODE,BASEMENTAREA_AVG,BASEMENTAREA_MEDI,BASEMENTAREA_MODE,CNT_CHILDREN,CNT_FAM_MEMBERS,CODE_GENDER,COMMONAREA_AVG,COMMONAREA_MEDI,COMMONAREA_MODE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_ID_PUBLISH,DAYS_LAST_PHONE_CHANGE,DAYS_REGISTRATION,DEF_30_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,ELEVATORS_AVG,ELEVATORS_MEDI,ELEVATORS_MODE,EMERGENCYSTATE_MODE,ENTRANCES_AVG,ENTRANCES_MEDI,ENTRANCES_MODE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,FLAG_CONT_MOBILE,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_2,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_EMAIL,FLAG_EMP_PHONE,FLAG_MOBIL,FLAG_OWN_CAR,FLAG_OWN_REALTY,FLAG_PHONE,FLAG_WORK_PHONE,FLOORSMAX_AVG,FLOORSMAX_MEDI,FLOORSMAX_MODE,FLOORSMIN_AVG,FLOORSMIN_MEDI,FLOORSMIN_MODE,FONDKAPREMONT_MODE,HOUR_APPR_PROCESS_START,HOUSETYPE_MODE,LANDAREA_AVG,LANDAREA_MEDI,LANDAREA_MODE,LIVE_CITY_NOT_WORK_CITY,LIVE_REGION_NOT_WORK_REGION,LIVINGAPARTMENTS_AVG,LIVINGAPARTMENTS_MEDI,LIVINGAPARTMENTS_MODE,LIVINGAREA_AVG,LIVINGAREA_MEDI,LIVINGAREA_MODE,NAME_CONTRACT_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,NAME_INCOME_TYPE,NAME_TYPE_SUITE,NONLIVINGAPARTMENTS_AVG,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_AVG,NONLIVINGAREA_MEDI,NONLIVINGAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,OCCUPATION_TYPE,ORGANIZATION_TYPE,OWN_CAR_AGE,REGION_POPULATION_RELATIVE,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,SK_ID_CURR,TARGET,TOTALAREA_MODE,WALLSMATERIAL_MODE,WEEKDAY_APPR_PROCESS_START,YEARS_BEGINEXPLUATATION_AVG,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_AVG,YEARS_BUILD_MEDI,YEARS_BUILD_MODE
0,24700.5,406597.5,351000.0,202500.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0247,0.0250,0.0252,0.0369,0.0369,0.0383,0,1.0,M,0.0143,0.0144,0.0144,-9461,-637,-2120,-1134.0,-3648.0,2.0,2.0,0.00,0.00,0.0000,No,0.0690,0.0690,0.0690,0.083037,0.262949,0.139376,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,N,Y,1,0,0.0833,0.0833,0.0833,0.1250,0.1250,0.1250,reg oper account,10,block of flats,0.0369,0.0375,0.0377,0,0,0.0202,0.0205,0.022,0.0190,0.0193,0.0198,Cash loans,Secondary / secondary special,Single / not married,House / apartment,Working,Unaccompanied,0.0000,0.0000,0.0,0.0000,0.00,0.0,2.0,2.0,Laborers,Business Entity Type 3,NaN,0.018801,2,2,0,0,0,0,100002,1.0,0.0149,"Stone, brick",WEDNESDAY,0.9722,0.9722,0.9722,0.6192,0.6243,0.6341
1,35698.5,1293502.5,1129500.0,270000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0959,0.0968,0.0924,0.0529,0.0529,0.0538,0,2.0,F,0.0605,0.0608,0.0497,-16765,-1188,-291,-828.0,-1186.0,0.0,0.0,0.08,0.08,0.0806,No,0.0345,0.0345,0.0345,0.311267,0.622246,NaN,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,N,N,1,0,0.2917,0.2917,0.2917,0.3333,0.3333,0.3333,reg oper account,11,block of flats,0.0130,0.0132,0.0128,0,0,0.0773,0.0787,0.079,0.0549,0.0558,0.0554,Cash loans,Higher education,Married,House / apartment,State servant,Family,0.0039,0.0039,0.0,0.0098,0.01,0.0,1.0,1.0,Core staff,School,NaN,0.003541,1,1,0,0,0,0,100003,0.0,0.0714,Block,MONDAY,0.9851,0.9851,0.9851,0.7960,0.7987,0.8040
2,6750.0,135000.0,135000.0,67500.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0,1.0,M,NaN,NaN,NaN,-19046,-225,-2531,-815.0,-4260.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.555912,0.729567,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,Y,Y,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,Revolving loans,Secondary / secondary special,Single / not married,House / apa

In [39]:
df = drop_null_columns(df)
df = drop_corr_columns(df)
df = drop_single_value_columns(df)

In [40]:
df.shape

(356255, 95)

In [41]:
d, df, obj_cols = encode(df, 50)

In [42]:
def get_nans_HC(df):
    """Создание словаря с заполнением пропусков"""
    df = df.copy()

    d = {}
    for col in df.columns:
        if col == "TARGET":
            continue
        d[col] = df[col].median()
    return d

In [43]:
nans = get_nans_HC(df)

In [44]:
def make_data_HC(df, nans):
    """Подготавливает данные в зависимости от классификатора"""
    df = df.copy()
    for col in df.columns:
        if col == "TARGET":
            continue
        df[col] = df[col].fillna(nans[col])
    return df

In [45]:
df = make_data_HC(df,nans)

In [46]:
def scale_HC(df):
    """Подготавливает данные в зависимости от классификатора"""
    df = df.copy()
    
    for col in df.columns:
        if col == "TARGET":
            continue
        df[col] = scale(df[col])
    return pd.DataFrame(df)

In [47]:
df = scale_HC(df)

/Users/ilyaboytsov/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


In [48]:
df = df[:100]

In [26]:
df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,LANDAREA_AVG,LIVINGAREA_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,ELEVATORS_MODE,ENTRANCES_MODE,LANDAREA_MODE,LIVINGAREA_MODE,NONLIVINGAREA_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,-1.733423,1,-0.324395,1.388015,-0.717914,-0.664531,-0.577538,0.142129,-0.478095,-0.166143,-0.398011,-0.829197,-0.570906,0.216852,-0.307758,-0.149452,1.506880,-0.456215,0.379837,0.579154,-0.146313,0.001803,0.468697,-0.499013,0.043245,1.599337,-0.245215,-0.974243,-1.265722,-0.103064,-0.062699,-0.838256,-0.631821,-0.124004,-0.231267,-0.205869,-0.291208,-0.547236,-0.467814,-0.709973,-3.021877,-1.317940,-2.153651,-1.002379,-0.831647,-0.174890,-2.059864,-0.368513,-1.056903,-1.061562,-0.354333,-0.901454,-0.305255,-0.954015,-0.762148,-0.152958,-0.355991,-1.017906,-0.299968,-0.864156,-0.260796,-0.08915,-0.902008,0.100446,-0.08734,0.242861,4.163504,5.253260,-0.206992,-0.006502,0.639065,-0.009017,-0.123882,-0.310738,-0.013853,-0.297632,-0.062538,-0.004771,-0.062669,-0.00255,-0.059477,-0.054269,-0.034802,-0.100138,-0.016332,-0.090534,-0.024402,-0.022529,-0.018305,-0.070987,-0.058766,-0.155837,-0.269947,-0.30862,-0.440926
1,-1.733413,0,-0.324395,-0.720370,-0.717914,1.504820,-0.577538,0.426792,1.725450,0.592683,1.029710,2.258774,1.061891,-0.628912,-0.307758,-1.252750,-0.166821,-0.460115,1.078697,1.790855,-0.146313,0.001803,0.468697,-0.499013,0.043245,1.599337,-0.245215,-0.283302,-0.167638,-2.067573,-2.051813,-0.284289,-0.325620,-0.124004,-0.231267,-0.205869,-0.291208,-0.547236,-0.467814,0.047052,-1.384737,0.564482,0.112063,-0.082629,-0.532205,0.129151,0.636080,0.431106,-1.544566,0.890461,-0.809463,-0.451266,-0.101705,-0.084008,-0.478647,0.125838,0.468762,-1.502149,-0.770249,-0.422615,-0.260796,-0.08915,-0.187810,1.391005,-0.08734,-0.174085,-0.320480,-0.275663,0.163107,-0.006502,0.639065,-0.009017,-0.123882,-0.310738,-0.013853,-0.297632,-0.062538,-0.004771,-0.062669,-0.00255,-0.059477,-0.054269,-0.034802,-0.100138,-0.016332,-0.090534,-0.024402,-0.022529,-0.018305,-0.070987,-0.058766,-0.155837,-0.269947,-0.30862,-1.007331
2,-1.733403,0,3.082659,1.388015,1.392925,-0.664531,-0.577538,-0.427196,-1.152888,-1.404669,-0.398011,-0.829197,-0.570906,0.216852,-0.307758,-0.783451,-0.689509,-0.453299,0.206116,0.306869,2.242932,0.001803,0.468697,2.003956,0.043245,1.599337,-0.245215,-0.974243,-1.265722,-0.103064,-0.062699,-0.284289,-0.938022,-0.124004,-0.231267,-0.205869,-0.291208,-0.547236,-0.467814,-0.061095,0.012103,0.216948,1.223975,-0.189847,-0.094270,0.046659,0.013939,-0.3685

In [49]:
# here is unsupervised implementation - i used it in order to make algorithm faster. if we change NearestNeighbors to
#kneighborsclassifier we will get supervised which is better
#from sklearn.neighbors import KNeighborsClassifier

from sklearn.neighbors import NearestNeighbors
def make_unsupervised_knn_HC(df, N_NEIGHBORS):
    """создает колонки с расстоянием от исходной точки до k-го соседа"""
    df = df.drop("TARGET",1)
    model = NearestNeighbors(n_neighbors=N_NEIGHBORS, algorithm = 'ball_tree',n_jobs = -1) # ball tree works faster, for better results use 'auto'
    model.fit(df)
    k_distances, indices = model.kneighbors(df)
    
    for i in range(1, N_NEIGHBORS):
        print(i)
        df["dist_{}_neigh".format(i)] = k_distances[:, i]
    df.to_csv("knn_dataset.csv")
    return df

In [50]:
unsup_df = make_unsupervised_knn_HC(df,3)

1
2


In [51]:
unsup_df.head()

,AMT_ANNUITY,AMT_CREDIT,AMT_INCOME_TOTAL,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_YEAR,APARTMENTS_AVG,APARTMENTS_MODE,BASEMENTAREA_AVG,BASEMENTAREA_MODE,CNT_CHILDREN,CNT_FAM_MEMBERS,CODE_GENDER,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_ID_PUBLISH,DAYS_LAST_PHONE_CHANGE,DAYS_REGISTRATION,DEF_30_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,ELEVATORS_AVG,ELEVATORS_MODE,EMERGENCYSTATE_MODE,ENTRANCES_AVG,ENTRANCES_MODE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,FLAG_CONT_MOBILE,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_2,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_EMAIL,FLAG_EMP_PHONE,FLAG_MOBIL,FLAG_OWN_CAR,FLAG_OWN_REALTY,FLAG_PHONE,FLAG_WORK_PHONE,FLOORSMAX_AVG,HOUR_APPR_PROCESS_START,HOUSETYPE_MODE,LANDAREA_AVG,LANDAREA_MODE,LIVE_CITY_NOT_WORK_CITY,LIVE_REGION_NOT_WORK_REGION,LIVINGAREA_AVG,LIVINGAREA_MODE,NAME_CONTRACT_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,NAME_INCOME_TYPE,NAME_TYPE_SUITE,NONLIVINGAREA_AVG,NONLIVINGAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,OCCUPATION_TYPE,ORGANIZATION_TYPE,OWN_CAR_AGE,REGION_POPULATION_RELATIVE,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,SK_ID_CURR,TOTALAREA_MODE,WALLSMATERIAL_MODE,WEEKDAY_APPR_PROCESS_START,YEARS_BEGINEXPLUATATION_AVG,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_AVG,dist_1_neigh,dist_2_neigh
0,-0.184959,-0.454489,0.144890,-0.056073,-0.067785,-0.250749,-0.356532,-0.145657,-0.447809,-1.001292,-0.949428,-0.834959,-0.767400,-0.575137,-1.268658,1.393136,1.509648,-0.458375,0.581113,-0.186118,0.378681,4.072560,5.166742,-0.371403,-0.358667,-0.087127,-1.055424,-1.016153,-2.970296,-1.329389,-2.148461,0.042819,-0.004433,-0.059576,-0.002369,-0.055245,-0.05041,-0.032331,-0.092972,-0.015173,-0.085343,-0.02267,-0.006041,-0.02093,-0.017006,0.622831,-0.009177,-0.123675,-0.310585,-0.013086,-0.299558,-0.063193,-0.276899,0.470903,0.002369,-0.717262,-0.665209,1.609105,-0.500154,-1.061198,-0.629137,-0.089662,-0.355793,-0.301532,-0.466654,-0.206365,-0.901862,-0.862676,-0.301678,-0.573794,0.224998,-0.30619,-0.832140,-0.394278,-0.305467,-0.261130,0.222727,-0.974361,-0.708788,-0.145259,-0.152069,-0.098848,-0.057337,-0.291012,-0.546013,-0.126086,-0.232707,-1.732036,-0.901267,0.103171,-0.826981,-0.180025,-0.157693,-2.053878,9.145585,9.181578
1,0.561576,1.770432,0.446895,-0.056073,-0.067785,-0.250749,-0.356532,-0.145657,-1.014764,-0.089804,-0.088399,-0.535958,-0.483939,-0.575137,-0.167256,-0.717733,-0.166044,-0.462264,1.786068,0.180322,1.076732,-0.313681,-0.271657,0.421865,0.458957,-0.087127,-1.541256,-1.498507,-1.360851,0.565767,0.111323,0.042819,-0.004433,-0.059576,-0.002369,-0.055245,-0.05041,-0.032331,-0.092972,-0.015173,-0.085343,-0.02267,-0.006041,-0.02093,-0.017006,0.622831,-0.009177,-0.123675,-0.310585,-0.013086,-0.299558,-0.063193,-0.276899,0.470903,0.002369,-0.717262,1.503287,1.609105,-0.500154,0.878414,-0.323099,-0.089662,-0.809292,-0.769804,-0.466654,-0.206365,-0.455381,-0.425140,-0.301678,1.060574,-0.624750,-0.30619,2.252847,1.047703,-0.103452,-0.261130,-0.162299,-0.282102,0.046545,-0.145259,-1.248730,-2.055999,-2.039162,-0.291012,-0.546013,-0.126086,-0.232707,-1.732026,-0.192795,1.390477,-0.271385,0.130041,0.126624,0.635585,7.251203,7.658356
2,-1.403424,-1.135828,-0.459119,-0.056073,-0.067785,-0.250749,-0.356532,-0.145657,-1.014764,-0.190939,-0.196028,-0.094930,-0.098067,-0.575137,-1.268658,1.393136,-0.689353,-0.455468,0.310344,0.195890,0.205160,-0.313681,-0.271657,-0.371403,-0.358667,-0.087127,-0.085169,-0.052844,0.013465,0.215882,1.235666,0.042819,-0.004433,-0.059576,-0.002369,-0.055245,-0.05041,-0.032331,-0.092972,-0.015173,-0.085343,